## Chapter 9: Frequent Pattern Mining
### Ex5: instacart_online_grocery_shopping_2017_05_01
Dataset: link download https://www.instacart.com/datasets/grocery-shopping-2017
(https://www.instacart.com/datasets/grocery-shopping-2017)
Requirement:
Read data
Pre-process data
Apply FPGrowth algorithm to find association rules from this dataset. Find the most popular
items in a basket.


In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
import findspark
findspark.init()

spark-2.4.0-bin-hadoop2.7/
spark-2.4.0-bin-hadoop2.7/python/
spark-2.4.0-bin-hadoop2.7/python/setup.cfg
spark-2.4.0-bin-hadoop2.7/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/resultiterable.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/shell.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/heapq3.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/join.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/version.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/rdd.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/java_gateway.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/find_spark_home.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/_globals.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/worker.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/accumulators.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/feature.py
spark-2.4.0-bin-hadoop2.7/python/pyspark

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
%cd '/content/gdrive/My Drive/LDS9_K273_ONLINE_Đinh Viết Trung'

/content/gdrive/My Drive/LDS9_K273_ONLINE_Đinh Viết Trung


In [4]:
# import libraries
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
from pyspark.sql.functions import mean, stddev, col, log
from pyspark.sql.functions import to_date, dayofweek, to_timestamp
from pyspark.sql import types 
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType
from pyspark.sql.functions import year, month
from pyspark.sql.functions import dayofmonth, weekofyear
from pyspark.sql.functions import split, explode
from pyspark.sql.functions import coalesce, first, lit
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import Bucketizer
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.sql.functions import regexp_extract, col
from pyspark.sql.functions import datediff
from pyspark.sql.functions import when

In [5]:
from pyspark import SparkContext
sc = SparkContext()

In [6]:
spark = SparkSession(sc)

In [8]:
# Loads data.
data = spark.read.csv('data/order_products__train.csv', 
                        header=True,
                        inferSchema=True)
data.count()


1384617

In [9]:
data.show(5)

+--------+----------+-----------------+---------+
|order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
|       1|     49302|                1|        1|
|       1|     11109|                2|        1|
|       1|     10246|                3|        0|
|       1|     49683|                4|        0|
|       1|     43633|                5|        1|
+--------+----------+-----------------+---------+
only showing top 5 rows



In [10]:
# Pre-processing data
from pyspark.sql.functions import collect_list, col, count, collect_set

data.createOrReplaceTempView("order_products_train")
products = spark.sql("select distinct product_id from order_products_train")
products.count()

39123

In [11]:
rawData = spark.sql("select * from order_products_train")
baskets = rawData.groupBy('order_id').agg(collect_set('product_id')\
 .alias('items'))
baskets.createOrReplaceTempView('baskets')


In [12]:
baskets.show(5, truncate=False)


+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|order_id|items                                                                                                                                                                                                               |
+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1342    |[30827, 3798, 14966, 21137, 46129, 33081, 13176, 7862]                                                                                                                                                              |
|1591    |[48246, 44116, 24852, 5194, 9130, 48823, 46473, 40310, 32520, 22105, 16900, 27681, 4103, 44008

In [14]:
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="items", minSupport=0.003, minConfidence=0.003)
model = fpGrowth.fit(baskets)


In [15]:
# Display frequent itemsets.
model.freqItemsets.show()


+--------------------+-----+
|               items| freq|
+--------------------+-----+
|             [13629]|  772|
|              [5194]|  475|
|             [24852]|18726|
|             [13176]|15480|
|             [35921]|  769|
|             [20345]|  473|
|             [21137]|10894|
|      [21137, 13176]| 3074|
|      [21137, 24852]| 2174|
|             [23165]|  764|
|             [13380]|  473|
|              [7969]|  472|
|             [21903]| 9784|
|      [21903, 21137]| 1639|
|[21903, 21137, 13...|  587|
|      [21903, 13176]| 2236|
|      [21903, 24852]| 2000|
|             [32478]|  763|
|             [47626]| 8135|
|      [47626, 21137]| 1017|
+--------------------+-----+
only showing top 20 rows



In [16]:
# transform examines the input items against all the association rules and summar
# consequents as prediction
mostPopularItemInABasket = model.transform(baskets)

In [17]:
mostPopularItemInABasket.show()


+--------+--------------------+--------------------+
|order_id|               items|          prediction|
+--------+--------------------+--------------------+
|    1342|[30827, 3798, 149...|[21903, 47626, 47...|
|    1591|[48246, 44116, 24...|[21137, 21903, 47...|
|    4519|             [29270]|                  []|
|    4935|             [45190]|                  []|
|    6357|[33731, 14669, 43...|[21137, 21903, 47...|
|   10362|[28522, 43789, 12...|[21137, 47626, 47...|
|   19204|[45255, 37285, 48...|                  []|
|   29601|[2716, 48057, 219...|[21137, 21903, 47...|
|   31035|[40723, 8174, 131...|[21137, 21903, 47...|
|   40011|[27292, 35213, 21...|[21137, 13176, 24...|
|   46266|[38558, 48642, 13...|[47626, 47766, 47...|
|   51607|[41390, 42752, 17...|                  []|
|   58797|[30827, 8803, 326...|[21137, 21903, 47...|
|   61793|[26348, 6184, 433...|[21137, 16797, 39...|
|   67089|[47766, 29388, 21...|[47626, 21137, 47...|
|   70863|[34791, 2618, 173...|      [13176, 2

### Use product_name instead of product_id

In [19]:
product_data = spark.read.csv('data/products.csv', header=True, inferSchema=True)


In [20]:
product_data.show(5, truncate=False)

+----------+-----------------------------------------------------------------+--------+-------------+
|product_id|product_name                                                     |aisle_id|department_id|
+----------+-----------------------------------------------------------------+--------+-------------+
|1         |Chocolate Sandwich Cookies                                       |61      |19           |
|2         |All-Seasons Salt                                                 |104     |13           |
|3         |Robust Golden Unsweetened Oolong Tea                             |94      |7            |
|4         |Smart Ones Classic Favorites Mini Rigatoni With Vodka Cream Sauce|38      |1            |
|5         |Green Chile Anytime Sauce                                        |5       |13           |
+----------+-----------------------------------------------------------------+--------+-------------+
only showing top 5 rows



In [21]:
product_data.createOrReplaceTempView("products")

rawData_1 = spark.sql('''select p.product_name, o.order_id from products p 
                    inner join order_products_train o 
                    where o.product_id = p.product_id''')

baskets_1 = rawData_1.groupBy('order_id').agg(collect_set('product_name')\
                                        .alias('items'))

baskets_1.createOrReplaceTempView('baskets')


In [22]:
baskets_1.head(3)

[Row(order_id=1342, items=['Raw Shrimp', 'Seedless Cucumbers', 'Versatile Stain Remover', 'Organic Strawberries', 'Organic Mandarins', 'Chicken Apple Sausage', 'Pink Lady Apples', 'Bag of Organic Bananas']),
 Row(order_id=1591, items=['Cracked Wheat', 'Strawberry Rhubarb Yoghurt', 'Organic Bunny Fruit Snacks Berry Patch', 'Goodness Grapeness Organic Juice Drink', 'Honey Graham Snacks', 'Spinach', 'Granny Smith Apples', 'Oven Roasted Turkey Breast', 'Pure Vanilla Extract', 'Chewy 25% Low Sugar Chocolate Chip Granola', 'Banana', 'Original Turkey Burgers Smoke Flavor Added', 'Twisted Tropical Tango Organic Juice Drink', 'Navel Oranges', 'Lower Sugar Instant Oatmeal  Variety', 'Ultra Thin Sliced Provolone Cheese', 'Natural Vanilla Ice Cream', 'Cinnamon Multigrain Cereal', 'Garlic', 'Goldfish Pretzel Baked Snack Crackers', 'Original Whole Grain Chips', 'Medium Scarlet Raspberries', 'Lemon Yogurt', 'Original Patties (100965) 12 Oz Breakfast', 'Nutty Bars', 'Strawberry Banana Smoothie', 'Gree

In [23]:
fpGrowth_1 = FPGrowth(itemsCol="items", 
                    minSupport=0.003, 
                    minConfidence=0.003)
model_1 = fpGrowth.fit(baskets_1)

# Display frequent itemsets.
model_1.freqItemsets.show(truncate=False)


+--------------------------------------------------------------------+-----+
|items                                                               |freq |
+--------------------------------------------------------------------+-----+
|[Organic Tomato Basil Pasta Sauce]                                  |772  |
|[Organic Spinach Bunch]                                             |475  |
|[Banana]                                                            |18726|
|[Bag of Organic Bananas]                                            |15480|
|[Organic Large Grade A Brown Eggs]                                  |769  |
|[Organic Blue Corn Tortilla Chips]                                  |473  |
|[Organic Strawberries]                                              |10894|
|[Organic Strawberries, Bag of Organic Bananas]                      |3074 |
|[Organic Strawberries, Banana]                                      |2174 |
|[Organic Leek]                                                      |764  |

In [24]:
mostPopularItemInABasket_1 = model_1.transform(baskets_1)

In [25]:
mostPopularItemInABasket_1.head(3)

[Row(order_id=1342, items=['Raw Shrimp', 'Seedless Cucumbers', 'Versatile Stain Remover', 'Organic Strawberries', 'Organic Mandarins', 'Chicken Apple Sausage', 'Pink Lady Apples', 'Bag of Organic Bananas'], prediction=['Organic Baby Spinach', 'Large Lemon', 'Organic Avocado', 'Organic Hass Avocado', 'Strawberries', 'Limes', 'Organic Raspberries', 'Organic Blueberries', 'Organic Whole Milk', 'Organic Cucumber', 'Organic Zucchini', 'Organic Yellow Onion', 'Organic Garlic', 'Seedless Red Grapes', 'Asparagus', 'Organic Grape Tomatoes', 'Organic Red Onion', 'Organic Baby Carrots', 'Honeycrisp Apple', 'Organic Cilantro', 'Organic Lemon', 'Sparkling Water Grapefruit', 'Raspberries', 'Organic Fuji Apple', 'Small Hass Avocado', 'Organic Baby Arugula', 'Organic Large Extra Fancy Fuji Apple', 'Original Hummus', 'Organic Blackberries', 'Organic Gala Apples', 'Fresh Cauliflower', 'Organic Half & Half', 'Michigan Organic Kale', 'Organic Small Bunch Celery', 'Organic Garnet Sweet Potato (Yam)', 'Orga

In [26]:
type(mostPopularItemInABasket_1)


pyspark.sql.dataframe.DataFrame

In [27]:
# chuyển list array thành string
from pyspark.sql.types import StringType

mostPopularItemInABasket_1.printSchema()


root
 |-- order_id: integer (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [28]:
mostPopularItemInABasket_1.createOrReplaceTempView("popular_items")

DF_cast = mostPopularItemInABasket_1.select('order_id', 
        mostPopularItemInABasket_1.items.cast(StringType()),
        mostPopularItemInABasket_1.prediction.cast(StringType())) 

In [29]:
DF_cast.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- items: string (nullable = true)
 |-- prediction: string (nullable = true)



In [30]:
mostPopularItemInABasket_1.createOrReplaceTempView("popular_items")

DF_cast = mostPopularItemInABasket_1.select('order_id', 
            mostPopularItemInABasket_1.items.cast(StringType()),
            mostPopularItemInABasket_1.prediction.cast(StringType())) 
DF_cast.printSchema()


root
 |-- order_id: integer (nullable = true)
 |-- items: string (nullable = true)
 |-- prediction: string (nullable = true)



In [31]:
DF_cast.head(3)

[Row(order_id=1342, items='[Raw Shrimp, Seedless Cucumbers, Versatile Stain Remover, Organic Strawberries, Organic Mandarins, Chicken Apple Sausage, Pink Lady Apples, Bag of Organic Bananas]', prediction="[Organic Baby Spinach, Large Lemon, Organic Avocado, Organic Hass Avocado, Strawberries, Limes, Organic Raspberries, Organic Blueberries, Organic Whole Milk, Organic Cucumber, Organic Zucchini, Organic Yellow Onion, Organic Garlic, Seedless Red Grapes, Asparagus, Organic Grape Tomatoes, Organic Red Onion, Organic Baby Carrots, Honeycrisp Apple, Organic Cilantro, Organic Lemon, Sparkling Water Grapefruit, Raspberries, Organic Fuji Apple, Small Hass Avocado, Organic Baby Arugula, Organic Large Extra Fancy Fuji Apple, Original Hummus, Organic Blackberries, Organic Gala Apples, Fresh Cauliflower, Organic Half & Half, Michigan Organic Kale, Organic Small Bunch Celery, Organic Garnet Sweet Potato (Yam), Organic Tomato Cluster, Carrots, Organic Peeled Whole Baby Carrots, Organic Italian Pars

In [32]:
DF_cast.write.csv('Chapter_9/Exercise5/mostPopularItemInABasket.csv')